## Setup and Import Libraries

In [9]:
import os
import pandas as pd
from typing import List
from langchain_core.documents import Document
from langchain_community.document_loaders import (
    CSVLoader, UnstructuredCSVLoader,
    UnstructuredExcelLoader
)

## Creating Directory and Saving Data

### Creating CSV Data

In [2]:
os.makedirs("data/structured_files", exist_ok=True)

In [4]:
data = {
    "Product": ["Laptop", "Mouse", "Keyboard", "Monitor", "Webcam"],
    "Category": ["Electronics", "Accessories", "Accessories", "Electronics", "Electronics"],
    "Price": [999.99, 29.99, 79.99, 299.99, 89.99],
    "Stock": [50, 200, 150, 75, 100],
    "Description": [
        "High-performance laptop with 16GB RAM and 512GB SSD",
        "Wireless optical mouse with ergonomic design",
        "Mechanical keyboard with RGB backlighting",
        "27-inch 4K monitor with HDR support",
        "1080p webcam with noise cancellation",
    ]
}

df = pd.DataFrame(data)
df.to_csv("data/structured_files/products.csv", index=False, header=True)

### Creating Excel Data

In [5]:
with pd.ExcelWriter("data/structured_files/inventory.xlsx") as writer:
    df.to_excel(excel_writer=writer, sheet_name="Products", index=False, header=True)

    # Add another sheet
    summary_data = {
        "Category": ["Electronics", "Accessories"],
        "Total_Items": [3, 2],
        "Total_Value": [1389.97, 109.98]
    }

    pd.DataFrame(data=summary_data).to_excel(
        excel_writer=writer,
        sheet_name="Summary",
        index=False,
        header=True
    )

## Processing CSV Files

### Processing CSV File

In [8]:
loader = CSVLoader(
    file_path="data/structured_files/products.csv",
    encoding="utf-8",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
    })

csv_documents = loader.load()

print(f"  Loaded {len(csv_documents)} documents (one per row)")
print("\nFirst Document")
print(f"Content Preview: {csv_documents[0].page_content[:100]}")
print(f"Metadata: {csv_documents[0].metadata}")

  Loaded 5 documents (one per row)

First Document
Content Preview: Product: Laptop
Category: Electronics
Price: 999.99
Stock: 50
Description: High-performance laptop w
Metadata: {'source': 'data/structured_files/products.csv', 'row': 0}


### Custom CSV Processing for Better Control

In [12]:
def process_csv_intelligently(file_path:str) -> List[Document]:
    """ 
    Process CSV with Intellient Document Creation
    """
    df = pd.read_csv(file_path)
    documents = []

    for idx, row in df.iterrows():
        # Create Structured Content
        content = f""" Product Information:
        Name: {row["Product"]}
        Category: {row["Category"]}
        Price: ${row["Price"]}
        Stock: {row["Stock"]} units
        Description: {row["Description"]}
        """

        doc = Document(
            page_content=content,
            metadata={
                "source": file_path,
                "row_index": idx,
                "product_name": row['Product'],
                "category": row["Category"],
                "price": row["Price"],
                "data_type": "product_info"
            }
        )

        documents.append(doc)

    return documents

In [13]:
documents = process_csv_intelligently(file_path="data/structured_files/products.csv")
documents

[Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_type': 'product_info'}, page_content=' Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99\n        Stock: 50 units\n        Description: High-performance laptop with 16GB RAM and 512GB SSD\n        '),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'price': 29.99, 'data_type': 'product_info'}, page_content=' Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99\n        Stock: 200 units\n        Description: Wireless optical mouse with ergonomic design\n        '),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'price': 79.99, 'data_type': 'product_info'}, page_co

## CSV Processing Strategies

In [15]:
print("\n📊 CSV Processing Strategies")
print("\n1. Row-based (CSVLoader)")
print("  ✅ Simple one-row-one-document")
print("  ✅ Good for record lookups")
print("  ❌ Loses table context")


print("\n2. Intelligent Processing")
print("  ✅ Preserves relationship")
print("  ✅ Create summaries")
print("  ✅ Rich metadata")
print("  ✅ Better for Q&A")



📊 CSV Processing Strategies

1. Row-based (CSVLoader)
  ✅ Simple one-row-one-document
  ✅ Good for record lookups
  ❌ Loses table context

2. Intelligent Processing
  ✅ Preserves relationship
  ✅ Create summaries
  ✅ Rich metadata
  ✅ Better for Q&A


## Processing Excel Files

In [16]:
def process_excel_with_pandas(file_path:str) -> List[Document]:
    """ 
    Process Excel with Sheet Awareness
    """
    excel_file = pd.ExcelFile(file_path)
    documents = []

    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Create Document for Each Sheet
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Column: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)
        

        doc = Document(
            page_content=sheet_content,
            metadata={
                "source": file_path,
                "sheet_name": sheet_name,
                "num_rows": len(df),
                "num_columns": len(df.columns),
                "data_type": "excel_sheet"
            }
        )

        documents.append(doc)

    return documents

In [18]:
excel_documents = process_excel_with_pandas(file_path="data/structured_files/inventory.xlsx")
excel_documents

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Products', 'num_rows': 5, 'num_columns': 5, 'data_type': 'excel_sheet'}, page_content='Sheet: Products\nColumn: Product, Category, Price, Stock, Description\nRows: 5\n\n Product    Category  Price  Stock                                         Description\n  Laptop Electronics 999.99     50 High-performance laptop with 16GB RAM and 512GB SSD\n   Mouse Accessories  29.99    200        Wireless optical mouse with ergonomic design\nKeyboard Accessories  79.99    150           Mechanical keyboard with RGB backlighting\n Monitor Electronics 299.99     75                 27-inch 4K monitor with HDR support\n  Webcam Electronics  89.99    100                1080p webcam with noise cancellation'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Summary', 'num_rows': 2, 'num_columns': 3, 'data_type': 'excel_sheet'}, page_content='Sheet: Summary\nColumn: Category, Total_Items

## Unstructured Excel Loader

In [21]:
excel_loader = UnstructuredExcelLoader(
    file_path="data/structured_files/inventory.xlsx",
    mode="elements"
)
unstructed_docs = excel_loader.load()

print("  ✅ Handles complex Excel features")
print("  ✅ Preserves formatting Info")
print("  ❌ Requires unstructued library")

  ✅ Handles complex Excel features
  ✅ Preserves formatting Info
  ❌ Requires unstructued library


In [22]:
unstructed_docs

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'file_directory': 'data/structured_files', 'filename': 'inventory.xlsx', 'last_modified': '2025-08-28T13:44:20', 'page_name': 'Products', 'page_number': 1, 'text_as_html': '<table><tr><td>Product</td><td>Category</td><td>Price</td><td>Stock</td><td>Description</td></tr><tr><td>Laptop</td><td>Electronics</td><td>999.99</td><td>50</td><td>High-performance laptop with 16GB RAM and 512GB SSD</td></tr><tr><td>Mouse</td><td>Accessories</td><td>29.99</td><td>200</td><td>Wireless optical mouse with ergonomic design</td></tr><tr><td>Keyboard</td><td>Accessories</td><td>79.99</td><td>150</td><td>Mechanical keyboard with RGB backlighting</td></tr><tr><td>Monitor</td><td>Electronics</td><td>299.99</td><td>75</td><td>27-inch 4K monitor with HDR support</td></tr><tr><td>Webcam</td><td>Electronics</td><td>89.99</td><td>100</td><td>1080p webcam with noise cancellation</td></tr></table>', 'languages': ['eng'], 'filetype': 'applicatio